In [40]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import pandas as pd
from pickle import dump, load

In [41]:
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

In [42]:
df = pd.DataFrame(columns=['word', 'embedding'])

,word,embedding


In [43]:
with open("../data/raw/dictionary_subset.txt", "r") as f:
    words = f.read().splitlines()

print(words)

for word in words:
    df_row = pd.DataFrame([[word, embed_model.get_text_embedding(word)]], columns=['word', 'embedding'])
    df = pd.concat([df, df_row], ignore_index=True)

['taking', 'takingly', 'takingness', 'takings', 'takins', 'takyr', 'takitumu', 'takkanah', 'takosis', 'takrouri', 'takt', 'taku', 'tal', 'tala', 'talabon', 'talahib', 'talaing', 'talayot', 'talayoti', 'talaje', 'talak', 'talalgia', 'talamanca', 'talamancan', 'talanton', 'talao', 'talapoin', 'talapoins', 'talar', 'talari', 'talaria', 'talaric', 'talars', 'talas', 'talbot', 'talbotype', 'talbotypist', 'talc', 'talced', 'talcer', 'talcher', 'talcing', 'talck', 'talcked', 'talcky', 'talcking', 'talclike', 'talcochlorite', 'talcoid', 'talcomicaceous', 'talcose', 'talcous', 'talcs', 'talcum', 'talcums', 'tald', 'tale', 'talebearer', 'talebearers', 'talebearing', 'talebook', 'talecarrier', 'talecarrying', 'taled', 'taleful', 'talegalla', 'talegallinae', 'talegallus', 'taleysim', 'talemaster', 'talemonger', 'talemongering', 'talent', 'talented', 'talenter', 'talenting', 'talentless', 'talents', 'talepyet', 'taler', 'talers', 'tales', 'talesman', 'talesmen', 'taleteller', 'taletelling', 'talewi

In [44]:
dump(df, open("../data/processed/embeddings.pkl", "wb"))